In [1]:
!wget 'https://raw.githubusercontent.com/Apress/applied-natural-language-processing-w-python/master/data_etc/deu.txt'

--2024-04-11 01:15:55--  https://raw.githubusercontent.com/Apress/applied-natural-language-processing-w-python/master/data_etc/deu.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12316220 (12M) [text/plain]
Saving to: 'deu.txt'

deu.txt             100%[===================>]  11.75M  --.-KB/s    in 0.09s   

2024-04-11 01:15:56 (131 MB/s) - 'deu.txt' saved [12316220/12316220]



In [2]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 77.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print(device)

cuda


In [6]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
import random
from torchtext.vocab import build_vocab_from_iterator
eng = spacy.load("en_core_web_sm") # Load the English model to tokenize English text
de = spacy.load("de_core_news_sm") # Load the German model to tokenize German text

In [7]:
import random

def split_dataset(input_file, train_file, test_file, train_percent):
    with open(input_file, 'r') as file:
        lines = file.readlines()  

    total_lines = len(lines)
    train_lines = int(total_lines * train_percent)
    test_lines = total_lines - train_lines


    random.shuffle(lines)

    with open(train_file, 'w') as train:
        for line in lines[:train_lines]:
            train.write(line)

    with open(test_file, 'w') as test:
        for line in lines[train_lines:]:
            test.write(line)


input_file = '/kaggle/working/deu.txt'


train_file = 'shiny_train.txt'
test_file = 'shiny_test.txt'

train_percent = 0.8

split_dataset(input_file, train_file, test_file, train_percent)

In [8]:
FILE_PATH = '/kaggle/working/shiny_train.txt'
data_pipe = dp.iter.IterableWrapper([FILE_PATH])
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)

In [9]:
for sample in data_pipe:
    print(sample)
    break

('Tom gave up on doing that.', 'Tom hat aufgegeben, das zu tun.')


In [10]:
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def deTokenize(text):
    """
    Tokenize a German text and return a list of tokens
    """
    return [token.text for token in de.tokenizer(text)]

In [11]:
print(engTokenize("Have a good day!!!"))
print(deTokenize("Haben Sie einen guten Tag!!!"))

['Have', 'a', 'good', 'day', '!', '!', '!']
['Haben', 'Sie', 'einen', 'guten', 'Tag', '!', '!', '!']


In [12]:
def getTokens(data_iter, place):
    """
    Function to yield tokens from an iterator. Since, our iterator contains
    tuple of sentences (source and target), `place` parameters defines for which
    index to return the tokens for. `place=0` for source and `place=1` for target
    """
    for english, german in data_iter:
        if place == 0:
            yield engTokenize(english)
        else:
            yield deTokenize(german)

In [13]:
source_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,0),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
source_vocab.set_default_index(source_vocab['<unk>'])

In [14]:
target_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,1),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
target_vocab.set_default_index(target_vocab['<unk>'])
    

In [15]:
print(source_vocab.get_itos()[:9])

['<pad>', '<sos>', '<eos>', '<unk>', '.', 'I', 'Tom', 'to', 'you']


In [16]:
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
        ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
        # 1 as seen in previous section
        T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
        # 2 as seen in previous section
        T.AddToken(2, begin=False)
    )
    return text_tranform

In [17]:
temp_list = list(data_pipe)
some_sentence = temp_list[798][0]
print("Some sentence= ", end="")
print(some_sentence)
transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))
print("Transformed sentence=", end="")
print(transformed_sentence)
index_to_string = source_vocab.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")

Some sentence= She cared for her sick mother.
Transformed sentence=[1, 50, 2890, 24, 70, 473, 232, 4, 2]
<sos> She cared for her sick mother . <eos> 

In [18]:
def applyTransform(sequence_pair):
    """
    Apply transforms to sequence of tokens in a sequence pair
    """

    return (
        getTransform(source_vocab)(engTokenize(sequence_pair[0])),
        getTransform(target_vocab)(deTokenize(sequence_pair[1]))
    )
data_pipe = data_pipe.map(applyTransform) ## Apply the function to each element in the iterator
temp_list = list(data_pipe)
print(temp_list[0])

([1, 6, 251, 67, 38, 182, 18, 4, 2], [1, 6, 18, 1858, 5, 14, 11, 77, 4, 2])


In [19]:
def sortBucket(bucket):
    """
    Function to sort a given bucket. Here, we want to sort based on the length of
    source and target sequence.
    """
    return sorted(bucket, key=lambda x: (len(x[0]), len(x[1])))


In [20]:
bucket = [
    (['how', 'are', 'you'], ['wie', 'geht', 'es', 'dir']),
    (['hello', 'world'], ['hallo', 'welt']),
    (['this', 'is', 'a', 'test'], ['das', 'ist', 'ein', 'test'])
]

sorted_data = sortBucket(bucket)

for item in sorted_data:
    print(item)

(['hello', 'world'], ['hallo', 'welt'])
(['how', 'are', 'you'], ['wie', 'geht', 'es', 'dir'])
(['this', 'is', 'a', 'test'], ['das', 'ist', 'ein', 'test'])


In [21]:
data_pipe = data_pipe.bucketbatch(
    batch_size = 64, batch_num=5,  bucket_num=1,
    use_in_batch_shuffle=False, sort_key=sortBucket
)

In [22]:
print(list(data_pipe)[0])

[([1, 3, 3, 37, 1010, 1567, 4867, 4, 2], [1, 3, 49, 107, 7667, 1398, 4, 2]), ([1, 1779, 17, 29, 446, 132, 684, 4, 2], [1, 1240, 2815, 1745, 2974, 671, 4, 2]), ([1, 35, 12, 1312, 23, 3378, 184, 4, 2], [1, 1790, 17, 9, 3201, 1086, 25, 2]), ([1, 40, 207, 12, 131, 132, 969, 4, 2], [1, 28, 65, 91, 348, 841, 4, 2]), ([1, 35, 8, 46, 142, 109, 595, 10, 2], [1, 555, 13, 59, 72, 609, 8, 2]), ([1, 27, 929, 1979, 13, 64, 1042, 4, 2], [1, 3620, 224, 10, 68, 420, 4, 2]), ([1, 59, 66, 8, 154, 112, 442, 4, 2], [1, 3765, 99, 13, 11, 578, 294, 4, 2]), ([1, 35, 12, 57, 16, 41, 149, 4, 2], [1, 1131, 12, 9, 20, 134, 202, 25, 2]), ([1, 5, 56, 271, 8, 11, 850, 4, 2], [1, 7, 82, 57, 21, 859, 1245, 4, 2]), ([1, 5, 43, 12, 291, 70, 334, 4, 2], [1, 7, 1046, 51, 9, 162, 329, 4, 2]), ([1, 40, 56, 77, 34, 178, 266, 4, 2], [1, 34, 65, 383, 136, 674, 56, 4, 2]), ([1, 26, 47, 355, 229, 55, 21, 4, 2], [1, 40, 470, 324, 136, 105, 31, 4, 2]), ([1, 143, 5, 76, 124, 7, 194, 10, 2], [1, 370, 15, 72, 11, 489, 323, 8, 2]), ([

In [23]:
def separateSourceTarget(sequence_pairs):
    """
    input of form: `[(X_1,y_1), (X_2,y_2), (X_3,y_3), (X_4,y_4)]`
    output of form: `((X_1,X_2,X_3,X_4), (y_1,y_2,y_3,y_4))`
    """
    sources,targets = zip(*sequence_pairs)
    return sources,targets

## Apply the function to each element in the iterator
data_pipe = data_pipe.map(separateSourceTarget)
print(list(data_pipe)[0])

(([1, 457, 21, 55, 30, 350, 4, 2], [1, 129, 405, 166, 9, 581, 4, 2], [1, 26, 22, 11, 333, 155, 4, 2], [1, 40, 14, 101, 98, 1859, 4, 2], [1, 26, 56, 97, 11, 3167, 4, 2], [1, 2768, 23, 193, 702, 23, 4, 2], [1, 201, 37, 41, 169, 1452, 10, 2], [1, 59, 13, 6, 14, 2596, 4, 2], [1, 36, 37, 533, 437, 3334, 4, 2], [1, 26, 85, 69, 2723, 21, 4, 2], [1, 5, 198, 9, 461, 335, 4, 2], [1, 59, 13, 88, 406, 3846, 4, 2], [1, 308, 3129, 13, 42, 3350, 4, 2], [1, 106, 8, 11, 83, 2228, 10, 2], [1, 6, 144, 23, 38, 66, 4, 2], [1, 26, 71, 34, 5290, 7543, 4, 2], [1, 5, 1220, 67, 112, 248, 4, 2], [1, 5, 168, 41, 1497, 1535, 4, 2], [1, 49, 14, 409, 14, 607, 4, 2], [1, 27, 532, 1260, 33, 432, 4, 2], [1, 159, 8, 174, 281, 622, 10, 2], [1, 5, 1049, 8, 47, 114, 4, 2], [1, 5, 45, 125, 170, 1901, 4, 2], [1, 26, 22, 88, 7752, 1407, 4, 2], [1, 36, 198, 749, 336, 519, 4, 2], [1, 6, 1297, 28, 11, 136, 4, 2], [1, 35, 12, 34, 98, 441, 4, 2], [1, 26, 85, 69, 514, 156, 4, 2], [1, 49, 1039, 15, 8, 44, 10, 2], [1, 26, 798, 21, 88

In [24]:
def applyPadding(pair_of_sequences):
    """
    Convert sequences to tensors and apply padding
    """
    return (T.ToTensor(0)(list(pair_of_sequences[0])), T.ToTensor(0)(list(pair_of_sequences[1])))
## `T.ToTensor(0)` returns a transform that converts the sequence to `torch.tensor` and also applies
# padding. Here, `0` is passed to the constructor to specify the index of the `<pad>` token in the
# vocabulary.
data_pipe = data_pipe.map(applyPadding)

In [25]:
print(list(data_pipe)[0])

(tensor([[   1,   99,   37,  ...,    0,    0,    0],
        [   1,  308,  494,  ...,    0,    0,    0],
        [   1,  106,    8,  ...,    0,    0,    0],
        ...,
        [   1,    6,  139,  ...,    0,    0,    0],
        [   1,    6,   52,  ...,    0,    0,    0],
        [   1,    6, 3065,  ...,  149,    4,    2]]), tensor([[   1,   34,  169,  ...,    0,    0,    0],
        [   1,  715,  593,  ...,    0,    0,    0],
        [   1,  263,   13,  ...,    0,    0,    0],
        ...,
        [   1,    6,  119,  ...,    0,    0,    0],
        [   1, 6768,   18,  ...,    0,    0,    0],
        [   1,    6, 2223,  ...,    3,    4,    2]]))


In [26]:
source_index_to_string = source_vocab.get_itos()
target_index_to_string = target_vocab.get_itos()

def showSomeTransformedSentences(data_pipe):
    """
    Function to show how the sentences look like after applying all transforms.
    Here we try to print actual words instead of corresponding index
    """
    for sources,targets in data_pipe:
        if sources[0][-1] != 0:
            continue # Just to visualize padding of shorter sentences
        for i in range(4):
            source = ""
            for token in sources[i]:
                source += " " + source_index_to_string[token]
            target = ""
            for token in targets[i]:
                target += " " + target_index_to_string[token]
            print(f"Source: {source}")
            print(f"Traget: {target}")
        break

showSomeTransformedSentences(data_pipe)

Source:  <sos> I do n't like to swim in the pool . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Traget:  <sos> Ich schwimme nicht gern im Becken . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Source:  <sos> A special tax is imposed on very high incomes . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Traget:  <sos> Sehr hohe Einkommen werden speziell <unk> . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Source:  <sos> The station is situated in between the two towns . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Traget:  <sos> Der Bahnhof liegt zwischen den zwei Städten . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Source:  <sos> The rich are not always happier than the poor . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Traget:  <sos> Reiche sind nicht immer glückli

In [27]:
for (data, target) in data_pipe:
    print(data)
    print(data.shape)
    print(target)
    print(target.shape)
    break

tensor([[    1,    49,    53,     8,    83,    48,    10,     2,     0],
        [    1,     6,    13,    11,   501,   990,     4,     2,     0],
        [    1,    26,    22,     7,    57,   121,     4,     2,     0],
        [    1,   543,  5744,    15,     8,   239,    10,     2,     0],
        [    1,     5,    46,     7,    95,   137,     4,     2,     0],
        [    1,     6,    13,   334,    24,     8,     4,     2,     0],
        [    1,   513,    21,   142,    19,   112,     4,     2,     0],
        [    1,    36,    97,   142,     3,  3141,     4,     2,     0],
        [    1,  2428,    13,   434,   501,   122,     4,     2,     0],
        [    1,    25,   218,     7,    34,   729,     4,     2,     0],
        [    1,   201,    92,   216,     8,  1485,    10,     2,     0],
        [    1,     5,    56,   120,    11,   187,     4,     2,     0],
        [    1,    26,    85,   647,     9,   133,     4,     2,     0],
        [    1,    26,    53,    42,   773,    60, 

In [28]:
import torch.nn as nn
import random
import torch.optim as optim
from tqdm import tqdm
import torch

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        
    def forward (self, x):
        #  sequence_lenght x batch_size 
        embedding = self.dropout(self.embedding(x))
        # embedding shape => (sequence_lenght,batch_size,embedding_size)
        outputs , (hidden, cell) = self.rnn(embedding)
        return hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size,
                num_layers, p):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hidden, cell):
        #shape of x: (batch_size) but we want (1, batch_size)
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1,batch_size, embedding_size)
        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape (1, N, hidden_size)
        predictions = self.fc(outputs)
        # predicitons shape: (1,batch_size, lenght_of_vocab)
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(target_vocab)
        
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        
        hidden, cell = self.encoder(source)
        # Grab start token
        x = target[0]
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            # output shape: (batch_size, target_vocab_size)
            outputs[t] = output
            best_guess = output.argmax(1)
            
            x = target[t] if random.random() < teacher_force_ratio else best_guess
        return outputs

In [31]:
# Training hyper parameters
num_epochs = 20
learning_rate = 0.001
batch_size = 64
load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(source_vocab)
output_size_decoder = len(target_vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5


In [32]:
encoder_net = Encoder(input_size_encoder, encoder_embedding_size, hidden_size,num_layers,
                    enc_dropout).to(device)
decoder_net = Decoder(output_size_decoder, decoder_embedding_size, hidden_size,output_size_decoder, num_layers,
                     dec_dropout).to(device)

In [33]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = target_vocab.lookup_indices(['<pad>'])[0]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [34]:
FILE_PATH = '/kaggle/working/shiny_test.txt'
test_data_pipe = dp.iter.IterableWrapper([FILE_PATH])
test_data_pipe = dp.iter.FileOpener(test_data_pipe, mode='rb')
test_data_pipe = test_data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)


In [35]:
import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, german_voc, english_voc, spacy_en, device, max_length=50):
    
    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_en(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
      
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, '<sos>')
    tokens.append('<eos>')
    # Go through each german token and convert to an index
    text_to_indices = english_voc.lookup_indices(tokens)

    
    # Convert to Tensor
    sentence_tensor = torch.tensor(text_to_indices, device=device).unsqueeze(1)
    
    # Build encoder hidden, cell state
    
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)
 
    outputs = german_voc.forward(['<sos>'])
    end_of_sentece = german_voc.forward(['<eos>'])[0]
    
    for _ in range(max_length):
        previous_word = torch.tensor([outputs[-1]], device=device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()
        outputs.append(best_guess)
    
    
        # Model predicts it's the end of the sentence
        if best_guess == end_of_sentece:
            break
    
    translated_sentence = german_voc.lookup_tokens(outputs)
    # remove start token
    return translated_sentence[1:]


In [36]:
from torchmetrics.functional.text import bleu_score
from torch.utils.data import DataLoader


def bleu(data_pipe, model, german_vocab, english_vocab, device):
    spacy_en = spacy.load("en_core_web_sm")
    targets = []
    outputs = []
    
    for idx, (source, target) in enumerate(data_pipe):
        if idx % 10000 == 0:
            print(idx)
        
        prediction = translate_sentence(model, source, german_vocab, english_vocab, spacy_en, device)
        prediction = prediction[:-1]
        detokenized  = ' '.join(deTokenize(target))
        targets.append([detokenized])
        outputs.append(" ".join(prediction))
#         print(f'Source: {source} Target: {t} Pred: {detokenized}', end='\n\n')
    return bleu_score(outputs, targets)

In [37]:
sentence = 'You must know yourself.'
spacy_en = spacy.load("en_core_web_sm")

for epoch in range(num_epochs):
    
    model.eval()
    translated_sentence = translate_sentence(model, sentence, target_vocab, source_vocab,spacy_en, device)
    print(' '.join(translated_sentence[:-1]))
    model.train()
    print(f'Epoch {epoch+1} / {num_epochs}')
    for batch_idx, (data, target) in enumerate(data_pipe):
            data = data.permute([1,0])
            data = data.to(device)
            target = target.permute([1,0])
            target = target.to(device)
            output = model(data, target)
            # output shape: (target_len, batch_size, output_len)
            output = output[1:].reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)
            
            optimizer.zero_grad()
            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()     

verzichten gestochen nächste nächste Telefonleitung Telefonleitung Telefonleitung dieselben Gruppenzwang Film Film irgendjemand entspannten Film träumte Inseln landete landete landete Entführung Maschinen Maschinen Maschinen Maschinen Maschinen zurückkommen zurückkommen anhören anhören Erfinder Erfinder Erfinder schaust landete landete landete landete Entführung Maschinen Maschinen Maschinen Maschinen Maschinen zurückkommen zurückkommen anhören anhören Erfinder Erfinder
Epoch 1 / 20
Du musst es .
Epoch 2 / 20
Du musst es wissen .
Epoch 3 / 20
Wir müssen es wissen .
Epoch 4 / 20
Du musst es selbst .
Epoch 5 / 20
Man muss es selbst wissen .
Epoch 6 / 20
Das musst ihr selbst wissen .
Epoch 7 / 20
Du musst es kennen .
Epoch 8 / 20
Das musst du selbst wissen .
Epoch 9 / 20
Das musst du selbst wissen .
Epoch 10 / 20
Man muss es selbst wissen .
Epoch 11 / 20
Man muss es selbst wissen .
Epoch 12 / 20
Man muss es selbst wissen .
Epoch 13 / 20
Man muss es selbst wissen .
Epoch 14 / 20
Man muss e

In [38]:
score = bleu(test_data_pipe, model, target_vocab, source_vocab, device)

0
10000
20000
30000


In [39]:
print(score)

tensor(0.0958)
